In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
"""
Read in the videos and play them
"""

from scale_cell_transport import files

video_dir = files.incucyte_video_dir_1()

paths = list(video_dir.glob("*.mp4"))

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

arrays = []
for path in tqdm(paths):
    cap = cv2.VideoCapture(str(path))
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    buffer = np.empty((n_frames, height, width, 3), dtype=np.uint8)

    fc = 0
    ret = True

    while fc < n_frames and ret:
        ret, buffer[fc] = cap.read()
        fc += 1

    # Convert BGR to RGB
    arrays.append(buffer[:, :, :, ::-1])
    cap.release()

In [ ]:
"""
Some of the videos are broken, so let's remove them from the list of videos and also the list of paths
"""

for i, (path, video) in enumerate(zip(paths, arrays)):
    if video.shape[:3] == (0, 0, 0):
        print(f"Removing {path}")
        paths.remove(path)
        arrays.pop(i)

In [ ]:
"""Display the first and second frame of each video"""

import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(arrays), 2, figsize=(5, 20))

for axs, video in zip(axes, arrays):
    for i in range(2):
        axs[i].axis("off")
        axs[i].imshow(video[i + 24])

In [ ]:
"""
Load the rotir model and weights
"""
import torch
from rotir.model import ImageRegistration

model_file = torch.load(files.model_path())

model = ImageRegistration(model_file["Parameter"]["model"])